# Project 2

Copyright 2017 Eamon and Lydia

Git ass

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [41]:
%matplotlib inline

from modsim import *

In [42]:
init = State(temp=82)
#ideal_temp = 57

In [76]:
coffee = System(init=init,
                volume=300,
                r=0.01,
                T_env=22,
                T_fridge=1.6,
                T_freezer=-18,
                t0=0,
                t_f_in=10,
                t_f_out_array=linrange(t_f_in, t_end, dt),
                t_end=30,
                dt=1)

In [77]:
def update_air(state, system):
    """update_air the thermal transfer model.
    
    state: State (temp)
    system: System object
    
    returns: State (temp)
    """
    unpack(system)
    T = state.temp
    T += -r * (T - T_env) * dt

    return State(temp=T)

In [78]:
def update_fridge(state, system):
    """update_air the thermal transfer model.
    
    state: State (temp)
    system: System object
    
    returns: State (temp)
    """
    unpack(system)
    T = state.temp
    T += -r * (T - T_fridge) * dt

    return State(temp=T)

In [79]:
def update_freezer(state, system):
    """update_air the thermal transfer model.
    
    state: State (temp)
    system: System object
    
    returns: State (temp)
    """
    unpack(system)
    T = state.temp
    T += -r * (T - T_freezer) * dt

    return State(temp=T)

In [80]:
def run_simulation(system, update1, update2, t_f_out):
    """Runs a simulation of the system.
    
    Add a TimeFrame to the System: results
    
    system: System object
    update_air_func: function that update_airs state
    """
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.loc[t0] = init
    ts = linrange(t0, t_end-dt, dt)
    
    for t in ts:
        if t < t_f_in:
            frame.loc[t+dt] = update1(frame.loc[t], system)
        else:
            if t < t_f_out:
                frame.loc[t+dt] = update2(frame.loc[t], system)
            else:
                frame.loc[t+dt] = update1(frame.loc[t], system)
    
    results = frame
    return results

In [81]:
def final_temp(system):
    """Final temperature.
    
    If system has no results, return initial temp.
    
    system: System object.
    
    returns: temperature (degC)
    """    
    if hasattr(system, 'results'):
        return system.results.temp[system.t_end]
    else:
        return 'No results'

In [82]:
 def sweep_simulation(system, update1, update2):
        sweep = SweepSeries()
        unpack(system)
        
        for time in t_f_out_array:
            t_f_out = time
            sim = run_simulation(system=system, update1=update1, update2=update2, t_f_out=t_f_out)
            sweep[time] = sim.temp[t_f_out]
        return sweep

Here's how we use it:

In [83]:
run_simulation(coffee, update_air, update_fridge, t_f_out = 15)
#final_temp(coffee)
coffee.t_f_out_array

array([ 10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.])

In [84]:
plot(coffee.results)

AttributeError: 'System' object has no attribute 'results'

In [85]:
run_simulation(coffee, update_air, update_freezer)
print(final_temp(coffee))
plot(coffee.results)

TypeError: run_simulation() missing 1 required positional argument: 't_f_out'

In [93]:
sweep1 = sweep_simulation(coffee, update_air, update_fridge)

In [94]:
plot(sweep1.index-t_f_in, sweep1.loc, color='purple')
decorate(xlabel='Time removed (min)',
         ylabel='Final temperature (C)',
         legend=False)

NotImplementedError: ix is not iterable